In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina' # Make visualizations look good
%matplotlib inline
#%config InlineBackend.figure_format = 'svg' 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import pickle as pkl
import dtreeviz
import imodelsx.process_results
from sklearn.tree import plot_tree
import sys
import pybaobabdt
import numpy as np
import viz
import llm_tree.data
import imodelsx
import sklearn.tree
sys.path.append('../experiments/')
results_dir = '/home/chansingh/mntv1/tree-prompt/mar18'

r = imodelsx.process_results.get_results_df(results_dir, use_cached=True)
r = r[r.seed == 1]

In [ ]:
# select a single model
rd = r
# rd = rd[rd.dataset_name == 'rotten_tomatoes']
rd = rd[rd.dataset_name == 'financial_phrasebank']
rd = rd[rd.model_name == 'manual_tree']
rd = rd[rd.checkpoint == 'EleutherAI/gpt-j-6B']
rd = rd[rd.verbalizer_num == 0]
rd = rd[rd.prompt_source == 'manual']
rd = rd[rd.num_prompts == 10]
display(rd.head())
run_args = rd.iloc[0]
model_name = run_args.model_name

### Load an individual model

In [ ]:
print('acc', run_args.accuracy_test, 'depth', run_args.max_depth, 'dset', run_args.dataset_name)
model = pkl.load(open(join(run_args.save_dir_unique, 'model.pkl'), 'rb'))

dt = model
feature_names = run_args.feature_names
feat_names = ['\n'.join(n.split(',')) for n in feature_names]
plt.figure(figsize=(8.5, 4.25))
sklearn.tree.plot_tree(dt, feature_names=feat_names, class_names=['neg', 'pos'],
                        impurity=False,
                        precision=2, rounded=True, proportion=True, filled=True, fontsize=5)
plt.savefig('tree_rt.pdf')
plt.show()

In [ ]:
def colors(x):
    if x == 0:
        return 'red'
    else:
        return 'blue'
# kwargs = {
#     'colormap': colors,
#     'custom_threshold': ['neg', 'pos'],
# }
# dt.classes_ = [b'neg', b'pos']
kwargs = {}

ax = pybaobabdt.drawTree(
    dt, figsize=(20, 10), dpi=300, ratio=0.55,
    features=feat_names, **kwargs
)
plt.savefig('tree_rt_bao.pdf')